In [199]:
PIANO_GENERATOR_MODEL_PATH = 'piano_model'
PIANO_GENERATOR_MODEL_WEIGHTS_PATH = './piano models/piano_model_vae_weights'

In [24]:
import os

import glob
import pickle
import numpy
import pandas
import numpy as np
from itertools import chain
from pydub import AudioSegment
from music21 import converter, instrument, note, chord, stream
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from tensorflow import keras
from keras import layers

In [3]:
def get_notes(path):
    """ Get all the notes and chords from the midi files in the 'path' directory """
    notes = []

    print(len(glob.glob(path + '/*.mid')))
    for file in glob.glob(path + '/*.mid'):
        midi = converter.parse(file)

#         print('parsing %s' % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [4]:
def get_notes(path):
    """ Get all the notes and chords from the midi files in the 'path' directory """
    notes = []

    print(len(glob.glob(path + '/*.mid')))
    for file in glob.glob(path + '/*.mid'):
        midi = converter.parse(file)

#         print('parsing %s' % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [148]:
def all_idx(idx, axis):
    grid = np.ogrid[tuple(map(slice, idx.shape))]
    grid.insert(axis, idx)
    return tuple(grid)

In [149]:
def onehot_initialization(a, ncols):
    out = np.zeros(a.shape + (ncols,), dtype=int)
    out[all_idx(a, axis=2)] = 1
    return out

In [161]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    ncols = max(max(network_input))+1
    ncols = ncols + (ncols%4)
    network_input = onehot_initialization(np.array(network_input), ncols)
    return network_input

In [162]:
notes = get_notes('midi_songs')

93


In [163]:
n_vocab = len(set(list(chain.from_iterable(notes))))
network_input = prepare_sequences(notes, n_vocab)

In [164]:
network_input.shape

(1868, 32, 256)

In [160]:
254%4

2

## Model - Variational Autoencoder

### Sampling layer

In [53]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

### Encoder

In [176]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(32, 256, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 32, 256, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 16, 128, 32)  320         input_39[0][0]                   
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 8, 64, 64)    18496       conv2d_15[0][0]                  
__________________________________________________________________________________________________
flatten_15 (Flatten)            (None, 32768)        0           conv2d_16[0][0]                  
____________________________________________________________________________________________

### Decoder

In [177]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(8 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((8, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_37 (Dense)             (None, 32768)             98304     
_________________________________________________________________
reshape_21 (Reshape)         (None, 8, 64, 64)         0         
_________________________________________________________________
conv2d_transpose_27 (Conv2DT (None, 16, 128, 64)       36928     
_________________________________________________________________
conv2d_transpose_28 (Conv2DT (None, 32, 256, 32)       18464     
_________________________________________________________________
conv2d_transpose_29 (Conv2DT (None, 32, 256, 1)        289       
Total params: 153,985
Trainable params: 153,985
Non-trainable params: 0
_____________________________________________________

### VAE Model

In [192]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(
                        data,
                        reconstruction
                    ), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [183]:
network_input.shape

(1868, 32, 256)

In [197]:
np.expand_dims(network_input, -1).astype("float32").shape

(1868, 32, 256, 1)

In [198]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(np.expand_dims(network_input, -1).astype("float32"), epochs=30, batch_size=32)

Epoch 1/30
59/59 [==============================] - 11s 172ms/step - loss: 2896.6780 - reconstruction_loss: 1236.9700 - kl_loss: 75.6030
Epoch 2/30
59/59 [==============================] - 10s 170ms/step - loss: 228.6087 - reconstruction_loss: 218.4733 - kl_loss: 4.8972
Epoch 3/30
59/59 [==============================] - 10s 173ms/step - loss: 213.9285 - reconstruction_loss: 207.7648 - kl_loss: 4.6891
Epoch 4/30
59/59 [==============================] - 10s 173ms/step - loss: 208.6802 - reconstruction_loss: 203.3631 - kl_loss: 4.1155
Epoch 5/30
59/59 [==============================] - 10s 170ms/step - loss: 204.0624 - reconstruction_loss: 199.5696 - kl_loss: 3.4288
Epoch 6/30
59/59 [==============================] - 10s 173ms/step - loss: 200.3169 - reconstruction_loss: 196.2312 - kl_loss: 2.7704
Epoch 7/30
59/59 [==============================] - 10s 177ms/step - loss: 196.8267 - reconstruction_loss: 194.9435 - kl_loss: 2.2175
Epoch 8/30
59/59 [==============================] - 11s 186

In [200]:
vae.save_weights(PIANO_GENERATOR_MODEL_WEIGHTS_PATH + '_01')